下载数据的命令

python collector.py download_data --source_dir ~/.qlib/fund_data/source/cn_1d --region CN  --delay 0.1 --interval 1d

# download from eastmoney.com
python collector.py download_data --source_dir ~/.qlib/fund_data/source/cn_1d --region CN --start 2020-11-01 --end 2020-11-10 --delay 0.1 --interval 1d

# normalize
python collector.py normalize_data --source_dir ~/.qlib/fund_data/source/cn_1d --normalize_dir ~/.qlib/fund_data/source/cn_1d_nor --region CN --interval 1d --date_field_name FSRQ

# dump data
cd qlib/scripts

python dump_bin.py dump_all --csv_path ~/.qlib/fund_data/source/cn_1d_nor --qlib_dir ~/.qlib/qlib_data/cn_fund_data --freq day --date_field_name FSRQ --include_fields DWJZ,LJJZ

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from read_and_write_fund_code import *
import datetime

# Step1： 读出来所有的数据

In [2]:
import qlib
from qlib.data import D

qlib.init(provider_uri="~/.qlib/qlib_data/cn_fund_data")
df = D.features(D.instruments(market="all"), ["$DWJZ", "$LJJZ"], freq="day")
df = df.reset_index()
df_new = df.copy()

[17486:MainThread](2021-07-25 12:10:55,988) INFO - qlib.Initialization - [config.py:276] - default_conf: client.
[17486:MainThread](2021-07-25 12:10:55,992) WARNING - qlib.Initialization - [config.py:292] - redis connection failed(host=127.0.0.1 port=6379), cache will not be used!
[17486:MainThread](2021-07-25 12:10:56,705) INFO - qlib.Initialization - [__init__.py:46] - qlib successfully initialized based on client settings.
[17486:MainThread](2021-07-25 12:10:56,707) INFO - qlib.Initialization - [__init__.py:47] - data_path=/Users/wangfan/.qlib/qlib_data/cn_fund_data


# Step2 数据加工

In [3]:
df_new["LJJZ_1d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-1)
df_new["LJJZ_5d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-5)
df_new["LJJZ_20d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-20)
df_new["LJJZ_60d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-60)
df_new["LJJZ_120d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-120)
df_new["LJJZ_240d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-240)
df_new["LJJZ_480d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-480)
df_new["LJJZ_720d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-720)
df_new.index = range(len(df_new))
df_new["y_1d"]=100 * (df_new["LJJZ_1d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_5d"]=100 * (df_new["LJJZ_5d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_20d"]=100 * (df_new["LJJZ_20d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_60d"]=100 * (df_new["LJJZ_60d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_120d"]=100 * (df_new["LJJZ_120d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_240d"]=100 * (df_new["LJJZ_240d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_480d"]=100 * (df_new["LJJZ_480d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_720d"]=100 * (df_new["LJJZ_720d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new = df_new.drop(columns = ['LJJZ_1d','LJJZ_5d','LJJZ_20d','LJJZ_60d','LJJZ_120d','LJJZ_240d','LJJZ_480d','LJJZ_720d'])

In [4]:
df_new["LJJZ_20d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(20)
df_new["LJJZ_60d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(60)
df_new["LJJZ_120d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(120)
df_new["LJJZ_240d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(240)
df_new["LJJZ_480d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(480)
df_new["LJJZ_720d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(720)
df_new.index = range(len(df_new))


df_new["his_20d"]=-100 * (df_new["LJJZ_20d"]- df_new["$LJJZ"])/df_new["LJJZ_20d"]
df_new["his_60d"]=-100 * (df_new["LJJZ_60d"]- df_new["$LJJZ"])/df_new["LJJZ_60d"]
df_new["his_120d"]=-100 * (df_new["LJJZ_120d"]- df_new["$LJJZ"])/df_new["LJJZ_120d"]
df_new["his_240d"]=-100 * (df_new["LJJZ_240d"]- df_new["$LJJZ"])/df_new["LJJZ_240d"]
df_new["his_480d"]=-100 * (df_new["LJJZ_480d"]- df_new["$LJJZ"])/df_new["LJJZ_480d"]
df_new["his_720d"]=-100 * (df_new["LJJZ_720d"]- df_new["$LJJZ"])/df_new["LJJZ_720d"]
df_new = df_new.drop(columns = ['LJJZ_20d','LJJZ_60d','LJJZ_120d','LJJZ_240d','LJJZ_480d','LJJZ_720d'])

In [5]:
df_train = df_new[df_new['y_720d'].notnull()]
df_test = df_new[df_new['y_720d'].isnull()]

In [6]:
df_train = df_train.dropna()

In [7]:
df_train["rank_20d"] = df_train.groupby(['datetime'])["his_20d"].rank(method='min',ascending=False)
df_train["rank_60d"] = df_train.groupby(['datetime'])["his_60d"].rank(method='min',ascending=False)
df_train["rank_120d"] = df_train.groupby(['datetime'])["his_120d"].rank(method='min',ascending=False)
df_train["rank_240d"] = df_train.groupby(['datetime'])["his_240d"].rank(method='min',ascending=False)
df_train["rank_480d"] = df_train.groupby(['datetime'])["his_480d"].rank(method='min',ascending=False)
df_train["rank_720d"] = df_train.groupby(['datetime'])["his_720d"].rank(method='min',ascending=False)
df_train.index = range(len(df_train))
# rank越小涨幅越大

In [8]:
w1, w2, w3, w4, w5, w6 = 6, 5, 4, 3, 2, 1

In [9]:
df_train["average_rank"] = df_train["rank_20d"] * w1 + df_train["rank_60d"] * w2 + df_train["rank_120d"] * w3 + df_train["rank_240d"] * w4 + df_train["rank_480d"] * w5 + df_train["rank_720d"] * w6
#df_train["average_rank"] = df_train["rank_120d"] + df_train["rank_240d"] + df_train["rank_480d"] + df_train["rank_720d"]
df_train["rank_of_average_rank"] = df_train.groupby(['datetime'])["average_rank"].rank(method='min')
df_train.index = range(len(df_train))
# rank越小，平均排名越高

In [10]:
df_train[(df_train['rank_of_average_rank']==1)&(df_train['datetime']=='2016-05-11')][['instrument','datetime','average_rank','rank_of_average_rank']]

,instrument,datetime,average_rank,rank_of_average_rank
1163599,320017,2016-05-11,1814.0,1.0


In [11]:
df_train[(df_train['rank_of_average_rank']==10)&(df_train['datetime']=='2016-05-11')][['instrument','datetime','average_rank','rank_of_average_rank']]

,instrument,datetime,average_rank,rank_of_average_rank
540272,160215,2016-05-11,4058.0,10.0


In [12]:
df_test["rank_20d"] = df_test.groupby(['datetime'])["his_20d"].rank(method='min',ascending=False)
df_test["rank_60d"] = df_test.groupby(['datetime'])["his_60d"].rank(method='min',ascending=False)
df_test["rank_120d"] = df_test.groupby(['datetime'])["his_120d"].rank(method='min',ascending=False)
df_test["rank_240d"] = df_test.groupby(['datetime'])["his_240d"].rank(method='min',ascending=False)
df_test["rank_480d"] = df_test.groupby(['datetime'])["his_480d"].rank(method='min',ascending=False)
df_test["rank_720d"] = df_test.groupby(['datetime'])["his_720d"].rank(method='min',ascending=False)
df_test.index = range(len(df_test))

/Users/wangfan/anaconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/wangfan/anaconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/wangfan/anaconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [13]:
df_test["average_rank"] = df_test["rank_20d"] * w1 + df_test["rank_60d"] * w2 + df_test["rank_120d"] * w3 + df_test["rank_240d"] * w4 + df_test["rank_480d"] * w5 + df_test["rank_720d"] * w6
df_test["rank_of_average_rank"] = df_test.groupby(['datetime'])["average_rank"].rank(method='min')
df_test.index = range(len(df_test))

/Users/wangfan/anaconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/wangfan/anaconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Step 3 模拟回测

In [14]:
df_train.columns

Index(['instrument', 'datetime', '$DWJZ', '$LJJZ', 'y_1d', 'y_5d', 'y_20d',
       'y_60d', 'y_120d', 'y_240d', 'y_480d', 'y_720d', 'his_20d', 'his_60d',
       'his_120d', 'his_240d', 'his_480d', 'his_720d', 'rank_20d', 'rank_60d',
       'rank_120d', 'rank_240d', 'rank_480d', 'rank_720d', 'average_rank',
       'rank_of_average_rank'],
      dtype='object')

In [15]:
def simulation_240d(start_rank,end_rank,start_date='1900-01-01',end_date='2021-04-29',dingtou=False,frequency=5):
    if not dingtou:
        return np.mean(df_train[(df_train['rank_of_average_rank']>=start_rank)
                &(df_train['rank_of_average_rank']<=end_rank)&(df_train['datetime']>start_date)&(df_train['datetime']<end_date)]['y_240d'])

In [16]:
start_rank=11
end_rank=15
start_date='1900-01-01'
end_date='2021-04-29'
df_train[(df_train['rank_of_average_rank']>=start_rank)
                &(df_train['rank_of_average_rank']<=end_rank)
                 &(df_train['datetime']>start_date)
                 &(df_train['datetime']<end_date)]

,instrument,datetime,$DWJZ,$LJJZ,y_1d,y_5d,y_20d,y_60d,y_120d,y_240d,...,his_480d,his_720d,rank_20d,rank_60d,rank_120d,rank_240d,rank_480d,rank_720d,average_rank,rank_of_average_rank
9,000001,2008-01-08,2.2670,3.4480,1.015084,2.233183,-2.871229,-11.397908,-15.081206,-23.810900,...,206.216690,212.036179,13.0,27.0,33.0,41.0,44.0,53.0,609.0,15.0
11,000001,2008-01-10,2.3390,3.5200,0.369319,-2.414774,-5.227273,-12.301140,-15.710226,-24.715904,...,215.695068,221.167908,14.0,14.0,33.0,37.0,43.0,51.0,534.0,14.0
12,000001,2008-01-11,2.3520,3.5330,0.198131,-1.981317,-6.057175,-14.690066,-16.161905,-24.568356,...,213.487137,223.831360,15.0,7.0,31.0,39.0,43.0,50.0,502.0,14.0
17,000001,2008-01-18,2.2820,3.4630,-2.252384,-1.414959,-4.187120,-15.737802,-16.142073,-23.678896,...,201.655045,210.304657,15.0,11.0,23.0,32.0,42.0,52.0,469.0,14.0
18,000001,2008-01-21,2.2040,3.3850,-3.161001,-1.358937,-3.190548,-13.825701,-13.648446,-21.004431,...,190.807541,203.315414,10.0,14.0,26.0,35.0,45.0,57.0,486.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783617,730001,2015-12-10,1.5770,2.3270,-0.859475,3.266017,-10.915339,-17.576277,-8.809623,-13.278901,...,116.666656,130.168137,59.0,155.0,399.0,20.0,46.0,127.0,3004.0,15.0
1788691,750005,2016-05-09,1.4750,1.4750,0.000000,0.067792,0.542374,4.000001,6.101697,7.925421,...,48.390343,46.039608,173.0,316.0,83.0,154.0,300.0,243.0,4255.0,14.0
1789342,762001,2016-02-17,1.6773,2.3073,-0.199369,1.304549,-0.988177,3.155200,9.378923,10.800496,...,106.858536,119.742874,299.0,195.0,248.0,14.0,10.0,30.0,3853.0,11.0
1789343,762001,2016-02-18,1.6727,2.3027,0.047773,-1.103060,0.047773,4.933340,8.737569,11.538623,...,104.575348,118.203362,121.0,231.0,269.0,12.0,9.0,26.0,3037.0,11.0


In [17]:
simulation_240d(1,10,'2012-01-01')

8.050922

In [18]:
simulation_240d(2,11,'2012-01-01')

8.165501

In [19]:
simulation_240d(1,5,'2012-01-01')

8.102766

In [20]:
simulation_240d(11,20,'2012-01-01')

7.6220393

In [21]:
simulation_240d(11,15,'2012-01-01')

7.6946383

In [22]:
simulation_240d(21,30,'2012-01-01')

7.8324203

# 2013以后

In [23]:
simulation_240d(1,10,'2013-01-01')

8.321775

In [24]:
simulation_240d(1,10,'2014-01-01')

8.470022

In [25]:
simulation_240d(1,10,'2015-01-01')

0.021299962

In [26]:
simulation_240d(1,10,'2016-01-01')

3.1755872

In [27]:
simulation_240d(1,10,'2017-01-01')

0.321701

# 定投

# Step 4 预测未来最优潜力组合

In [30]:
# 根据单月盈利排名选择
def get_list_by_rank(start_rank,end_rank,date='2021-04-29'): 
    df_left = df_test[(df_test['rank_of_average_rank']>=start_rank)&(df_test['rank_of_average_rank']<=end_rank)&(df_test['datetime']==date)][['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_fund_code = read_fund_code_from_DB()
    return pd.merge(df_left, df_fund_code, how='left', left_on='instrument', right_on='code')

get_list_by_rank(1,5, date='2021-04-29')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,005296,2021-04-29,2080.0,1.0,3875,005296,NHFCHHA,南华丰淳混合A,混合型-偏股
1,005297,2021-04-29,2202.0,2.0,3876,005297,NHFCHHC,南华丰淳混合C,混合型-偏股
2,090020,2021-04-29,2315.0,3.0,10774,090020,DCJKCYHH,大成健康产业混合,混合型-偏股
3,004040,2021-04-29,2612.0,4.0,2963,004040,JYYLJKCYA,金鹰医疗健康产业A,股票型
4,004041,2021-04-29,2772.0,5.0,2964,004041,JYYLJKCYC,金鹰医疗健康产业C,股票型


In [31]:
# 根据编码和日期选择
def get_list_by_no(no,date='2021-04-29'): 
    df_left = df_test[(df_test['instrument'] == no)&(df_test['datetime']==date)][['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_fund_code = read_fund_code_from_DB()
    return pd.merge(df_left, df_fund_code, how='left', left_on='instrument', right_on='code')
get_list_by_no(no='001532',date='2021-04-23')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,001532,2021-04-23,7339.0,21.0,1162,001532,HAWTJKHH,华安文体健康混合,混合型-灵活


## 机会识别
连续几个月的排名都在100名之内的基金: 
过去4个月都在前100，建议购买

In [59]:
topN = 100
date_list = ['2021-04-23','2021-05-24','2021-06-23','2021-07-23']
df_date_list = pd.DataFrame({'date':date_list})

def get_topN_for_months(df_test, topN,df_date_list):
    # condider Date
    df_date_list = pd.DataFrame({'date':date_list})
    df_date_list['date'] = pd.to_datetime(df_date_list['date'])
    df_left = df_test[['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_left = pd.merge(df_left, df_date_list, how='inner', left_on='datetime', right_on='date')


    # consider TopN
    df_max_rank = pd.DataFrame()
    df_max_rank["max_rank"] = df_left.groupby(['instrument'])["rank_of_average_rank"].max()
    df_max_rank = df_max_rank.reset_index()
    df_max_rank = df_max_rank[df_max_rank["max_rank"]<=topN]
    df_left = pd.merge(df_left, df_max_rank, how='inner', on='instrument')
    df_fund_code = read_fund_code_from_DB()
    return pd.merge(df_left, df_fund_code, how='left', left_on='instrument', right_on='code')

df_result = get_topN_for_months(df_test, topN,df_date_list)
df_result[df_result['date'] == '2021-07-23' ]

,instrument,datetime,average_rank,rank_of_average_rank,date,max_rank,index,code,ename,cname,type
1,001298,2021-07-23,722.0,4.0,2021-07-23,22.0,980,001298,JYMZXXHH,金鹰民族新兴混合,混合型-灵活
5,001532,2021-07-23,4066.0,62.0,2021-07-23,62.0,1162,001532,HAWTJKHH,华安文体健康混合,混合型-灵活
9,003567,2021-07-23,799.0,5.0,2021-07-23,61.0,2678,003567,HXHYJQHH,华夏行业景气混合,混合型-偏股
13,001487,2021-07-23,4542.0,69.0,2021-07-23,92.0,1128,001487,BYYSCYHHA,宝盈优势产业混合A,混合型-灵活
17,005977,2021-07-23,3844.0,55.0,2021-07-23,55.0,4462,005977,ZXBCZXHHA,中信保诚至兴混合A,混合型-灵活
21,005978,2021-07-23,4044.0,61.0,2021-07-23,61.0,4463,005978,ZXBCZXHHC,中信保诚至兴混合C,混合型-灵活
25,006049,2021-07-23,5667.0,85.0,2021-07-23,85.0,4521,006049,HYYJJXHHAB,恒越研究精选混合A/B,混合型-偏股
29,501057,2021-07-23,1370.0,13.0,2021-07-23,13.0,12148,501057,HTFZZXNYQCA,汇添富中证新能源汽车A,指数型-股票
33,501058,2021-07-23,1420.0,14.0,2021-07-23,14.0,12149,501058,HTFZZXNYQCC,汇添富中证新能源汽车C,指数型-股票


### 持仓中的表现情况，淘汰比较差的基金

In [63]:
my_fund_list = [
'002943',
'180012',
'001532',
'162605',
'163406',
'005233',
'000083',
'005827',
'008099',
'004997',
'164205',
'001557',
'502056',
'501057',
'000628',
'163417',
'001410',
'001298',
'003567',
'005167',
'003096','001632','004925']
df_no_list = pd.DataFrame({'no':my_fund_list})

date_list = ['2021-04-23','2021-05-24','2021-06-23','2021-07-23']

def get_list_by_no_list(df_test, df_no_list,date_list):   
    # condider Date
    df_date_list = pd.DataFrame({'date':date_list})
    df_date_list['date'] = pd.to_datetime(df_date_list['date'])
    df_left = df_test[['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_left = pd.merge(df_left, df_date_list, how='inner', left_on='datetime', right_on='date')
    
    df_left = df_left[['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_left = pd.merge(df_left, df_no_list, how='inner', left_on='instrument', right_on='no')
    
        # consider TopN
    df_max_rank = pd.DataFrame()
    df_max_rank["mean_rank"] = df_left.groupby(['instrument'])["rank_of_average_rank"].mean()
    df_max_rank = df_max_rank.reset_index().sort_values(by=['mean_rank'])
    
    df_fund_code = read_fund_code_from_DB()
    return pd.merge(df_max_rank, df_fund_code, how='left', left_on='instrument', right_on='code')

df_result = get_list_by_no_list(df_test,df_no_list,date_list)
df_result

,instrument,mean_rank,index,code,ename,cname,type
0,001298,8.25,980,001298,JYMZXXHH,金鹰民族新兴混合,混合型-灵活
1,501057,12.50,12148,501057,HTFZZXNYQCA,汇添富中证新能源汽车A,指数型-股票
2,003567,30.25,2678,003567,HXHYJQHH,华夏行业景气混合,混合型-偏股
3,001532,39.75,1162,001532,HAWTJKHH,华安文体健康混合,混合型-灵活
4,002943,138.00,2235,002943,GFDYZHH,广发多因子混合,混合型-灵活
5,004925,162.25,3574,004925,CXDTHBHYLHGP,长信低碳环保行业量化股票,股票型
6,180012,565.00,11610,180012,YHFYZTHH,银华富裕主题混合,混合型-偏股
7,001557,715.50,1183,001557,THZZ500ZSZQC,天弘中证500指数增强C,指数型-股票
8,003096,812.25,2334,003096,ZOYLJKHHC,中欧医疗健康混合C,混合型-偏股
9,001410,926.25,1071,001410,XDAYXNYCYGP,信达澳银新能源产业股票,股票型
